---
## 💾 Reading the channels from Nikon microscope images with `.nd2` format

In [ ]:
## requirements
import logging
logging.getLogger().setLevel(logging.INFO)

import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname,splitext
from os import makedirs

#!pip install nd2
import nd2 
import numpy as np

In [ ]:
## parameters
input_path=None
output_path=None
zstack=None

In [ ]:
## inferred parameters
assert dirname(output_path).endswith('channels')
output_dir_path=dirname(output_path)+'/'
logging.info(output_dir_path)

In [ ]:
f1 = nd2.ND2File(input_path)

## Image info

In [ ]:
print(('\n'+'-'*50+'\n').join([f"{k}\n{'-'*50}\n{v}" for k,v in f1.text_info.items()]))

### Dimensions

In [ ]:
print(f1.sizes)

### Channel names

In [ ]:
channels={d.channel.index:d.channel.name.lower() for d in f1.metadata.channels}
logging.info(channels)

### Dimension names

In [ ]:
print(f1.metadata.contents)

In [ ]:
## alt method for getting the lables
{getattr(f1.metadata.contents,s):s.replace('Count','') for s in dir(f1.metadata.contents) if not s.startswith('_')}

In [ ]:
## alt method for getting the lables
{getattr(f1.attributes,k):k.replace('Count','') for k in dir(f1.attributes) if not k.startswith('_') and k.endswith('Count') and not k.startswith('component')}

## Save the 2D arrays

In [ ]:
def save_channels(
    f1,
    output_dir_path,
    ):
    a1=f1.asarray()
    assert a1.shape[0]==f1.metadata.contents.channelCount==len(channels) and len(a1.shape)==3, a1.shape

    ## get the channel
    for i,k in channels.items():
        makedirs(
            output_dir_path,
            exist_ok=True,
            )
        ## slice an image
        a2=a1[zstack,i,:,:] if not zstack is None else a1[i,:,:]
        assert len(a2.shape)==2, a2.shape

        np.save(
            f"{output_dir_path}/{k}.npy",
             a2,
            )
        fig,ax=plt.subplots()
        ax.imshow(
            a2,
            vmin=100,
            vmax=200,
        )
        ax.set_title(f"channel={k}")    
    # break
    return a1

In [ ]:
if f1.metadata.contents.frameCount!=1 and zstack is None:
    import sys
    import logging
    logging.error(f"zstack found in {input_path} of contents. skipping the file. shape={f1.asarray().shape}")
    # sys.exit(0)
else:
    ## get the frames
    a1=save_channels(
    f1,
    output_dir_path,
    )